# Cross Validation and the Bootstrap

In [1]:
import numpy as np
import statsmodels.api as sm

from ISLP import load_data
from ISLP.models import (ModelSpec as MS , summarize, poly)
from ISLP.models import sklearn_sm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone 

from functools import partial


## The Validation Set Approach 
- Recall that the **Validaiton Set Approach** is just dividing the observations $n$ into two parts
- Training Set
- Validation Set 

In [3]:
Auto = load_data('Auto')
Auto_train, Auto_test = train_test_split(Auto,test_size=196,random_state=10)

- Loading the **Auto** data set which contain $392$
- Splitting the dataset into two equal parts 

In [11]:
hp_mm = MS(['horsepower'])

X_train = hp_mm.fit_transform(Auto_train)
Y_train = Auto_train['mpg']

simple_LR = sm.OLS(Y_train,X_train)
results = simple_LR.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,41.0730,1.031,39.825,0.0
horsepower,-0.1632,0.009,-17.913,0.0


- After fiting a **Simple Linear Regression** model using the **training set**
- We validate the model using the **test/Validate set** with the `predict()` method

In [18]:
X_test  = hp_mm.transform(Auto_test)
y_test = Auto_test['mpg']
valid_pred= results.predict(X_test)
np.mean((y_test-valid_pred)**2)

23.060588342506232

- The test **MSE** is estimated to be around $23.060$

In [ ]:
def evalMSE(terms,response,train,test):
    mm = MS(terms)
    X_train =mm.fit_transform(train)
    Y_train = train[response]